### HW7

Yingfeng Lou

### (a) Download the Anuran Calls (MFCCs) Data Set. Choose 70% of the data randomly as the training set.

In [60]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import hamming_loss,accuracy_score
import matplotlib.pyplot as plt

In [61]:
from imblearn.over_sampling import SMOTE

In [62]:
import warnings
warnings.filterwarnings('ignore') 

In [107]:
df=pd.read_csv('C:/Users/hp/Desktop/EE559/HW/Homework 7 Data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')
x,y=df.iloc[:,:22],df.iloc[:,22:-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
#print(df.shape)
#print(x_test)
print("size of data set",df.shape)
print("size of x_train",x_train.shape)
print("size of x_test",x_test.shape)
print("size of y_train",y_train.shape)
print("size of y_test",y_test.shape)

size of data set (7195, 26)
size of x_train (5036, 22)
size of x_test (2159, 22)
size of y_train (5036, 3)
size of y_test (2159, 3)


### (b) Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-class classification is to train a classifier for each label. We first try this approach:

### i. Research exact match and hamming score/ loss methods for evaluating multilabel classification and use them in evaluating the classifiers in this problem

Accuracy score/ Exact match metric:calculates subset accuracy meaning the predicted set of labels should exactly match with the true set of labels. I can use function:metrics.accuracy_score(y_true, y_pred) to find the accuracy classification score.  
Hamming loss: The fraction of the wrong labels to the total number of labels. I can use metrics.hamming_loss(y_true, y_pred) to compute the average Hamming loss.

### ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. You are welcome to try to solve the problem with both normalized and raw attributes and report the results

In [16]:
y_train_family, y_train_genus, y_train_species=y_train["Family"], y_train["Genus"], y_train["Species"]
y_test_family, y_test_genus, y_test_species=y_test["Family"], y_test["Genus"], y_test["Species"]

svm=SVC(kernel='rbf',C=10**(-3)).fit(x_train,y_train_family)
c=svm.score(x_train,y_train_family)
print("the acc when C=10^(-3),lamda=-3",c)
svm=SVC(kernel='rbf',C=10**3).fit(x_train,y_train_family)
c_1=svm.score(x_train,y_train_family)
print("the acc when C=10^(3),lamda=3",c_1)

the acc when C=10^(-3),lamda=-3 0.6102065131056394
the acc when C=10^(3),lamda=3 1.0


So I can choose lamda=[-3,3] i.e.C=[10^-3,10^3] to ensure the acc will not below 0.6102065131056394

In [9]:
def SvmModel(x_train,y_train_label):
    param = {'C':np.logspace(-3,3,7),'gamma':np.linspace(.1,2,10)}
    svm=SVC(kernel='rbf')
    svm_param=GridSearchCV(svm,param,cv=10)
    svm_param.fit(x_train,y_train_label)
    c=svm_param.best_params_['C']
    gamma=svm_param.best_params_['gamma']
    score=svm_param.best_score_        
    return c, gamma, score

In [10]:
y_train_family, y_train_genus, y_train_species=y_train["Family"], y_train["Genus"], y_train["Species"]
y_test_family, y_test_genus, y_test_species=y_test["Family"], y_test["Genus"], y_test["Species"]

c_family,gamma_family,score_family=SvmModel(x_train,y_train_family)
print("----------paramater of Family----------")
print("C_family=",c_family,"gamma_family=",gamma_family,"score_family=",score_family)
#print("scorescore_family)
c_genus,gamma_genus,score_genus=SvmModel(x_train,y_train_genus)
print("----------paramater of Genus----------")
print("C_genus=",c_genus,"gamma_genus=",gamma_genus,"score_genus=", score_genus)
c_species,gamma_species,score_species=SvmModel(x_train,y_train_species)
print("----------paramater of Species----------")
print("C_species=",c_species,"gamma_species=",gamma_species,"score_species=", score_species)

----------paramater of Family----------
C_family= 10.0 gamma_family= 2.0 score_family= 0.9934480419072864
----------paramater of Genus----------
C_genus= 100.0 gamma_genus= 1.577777777777778 score_genus= 0.990864337782827
----------paramater of Species----------
C_species= 10.0 gamma_species= 1.788888888888889 score_species= 0.9904675123860013


In [11]:
from sklearn.metrics import hamming_loss,accuracy_score

svm_family=SVC(kernel='rbf',C=c_family,gamma=gamma_family).fit(x_train,y_train_family)
y_predict_family=pd.Series(svm_family.predict(x_test))
hamming_loss_family =hamming_loss(y_predict_family, y_test_family)
exact_match_family= accuracy_score(y_test_family, y_predict_family)
print("----------Family---------")
print("hamming loss of Family:",hamming_loss_family)
print("exact match of Family:",exact_match_family)

svm_genus=SVC(kernel='rbf',C=c_genus,gamma=gamma_genus).fit(x_train,y_train_genus)
y_predict_genus=pd.Series(svm_genus.predict(x_test))
hamming_loss_genus =hamming_loss(y_predict_genus, y_test_genus)
exact_match_genus= accuracy_score(y_test_genus, y_predict_genus)
print("----------Genus---------")
print("hamming loss of genus:",hamming_loss_genus)
print("exact match of Genus:",exact_match_genus)

svm_species=SVC(kernel='rbf',C=c_species,gamma=gamma_species).fit(x_train,y_train_species)
y_predict_species=pd.Series(svm_species.predict(x_test))
hamming_loss_species =hamming_loss(y_predict_species, y_test_species)
exact_match_species= accuracy_score(y_test_species, y_predict_species)
print("----------Species---------")
print("hamming loss of species:",hamming_loss_species)
print("exact match of species:",exact_match_species)

----------Family---------
hamming loss of Family: 0.006021306160259379
exact match of Family: 0.9939786938397406
----------Genus---------
hamming loss of genus: 0.009263547938860583
exact match of Genus: 0.9907364520611394
----------Species---------
hamming loss of species: 0.009726725335803613
exact match of species: 0.9902732746641963


iii. Repeat 1(b)ii with L1-penalized SVMs. Remember to normalize the attributes.

The Dataset is already normalized, so I can use it directly.

In [12]:
from sklearn.svm import LinearSVC
def Svm_L1(x_train,y_train_label):
    param = {'C':np.logspace(-3,3,7)}
    svm=LinearSVC(penalty='l1', dual=False,random_state=42)
    svm_param=GridSearchCV(svm,param,cv=10)
    svm_param.fit(x_train,y_train_label)
    c=svm_param.best_params_['C']
    score=svm_param.best_score_
    return c, score

In [13]:
c_family,score_family=Svm_L1(x_train,y_train_family)
print("----------paramater of Family----------")
print("C_family=",c_family,"score_family=",score_family)
#print("scorescore_family)
c_genus,score_genus=Svm_L1(x_train,y_train_genus)
print("----------paramater of Genus----------")
print("C_genus=",c_genus,"score_genus=", score_genus)
c_species,score_species=Svm_L1(x_train,y_train_species)
print("----------paramater of Species----------")
print("C_species=",c_species,"score_species=", score_species)

----------paramater of Family----------
C_family= 100.0 score_family= 0.9408280475874911
----------paramater of Genus----------
C_genus= 100.0 score_genus= 0.9507565716810248
----------paramater of Species----------
C_species= 10.0 score_species= 0.959092666224873


In [15]:
svm_family=LinearSVC(penalty='l1', dual=False,C=c_family).fit(x_train,y_train_family)
y_predict_family=pd.Series(svm_family.predict(x_test))
hamming_loss_family =hamming_loss(y_predict_family, y_test_family)
exact_match_family= accuracy_score(y_test_family, y_predict_family)
print("----------Family---------")
print("hamming loss of Family:",hamming_loss_family)
print("exact match of Family:",exact_match_family)

svm_genus=LinearSVC(penalty='l1', dual=False,C=c_genus).fit(x_train,y_train_genus)
y_predict_genus=pd.Series(svm_genus.predict(x_test))
hamming_loss_genus =hamming_loss(y_predict_genus, y_test_genus)
exact_match_genus= accuracy_score(y_test_genus, y_predict_genus)
print("----------Genus---------")
print("hamming loss of genus:",hamming_loss_genus)
print("exact match of Genus:",exact_match_genus)

svm_species=LinearSVC(penalty='l1', dual=False,C=c_species).fit(x_train,y_train_species)
y_predict_species=pd.Series(svm_species.predict(x_test))
hamming_loss_species =hamming_loss(y_predict_species, y_test_species)
exact_match_species= accuracy_score(y_test_species, y_predict_species)
print("----------Species---------")
print("hamming loss of species:",hamming_loss_species)
print("exact match of species:",exact_match_species)

----------Family---------
hamming loss of Family: 0.07364520611394164
exact match of Family: 0.9263547938860583
----------Genus---------
hamming loss of genus: 0.05974988420565076
exact match of Genus: 0.9402501157943493
----------Species---------
hamming loss of species: 0.03844372394627142
exact match of species: 0.9615562760537286


iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

In [5]:
def Svm_Resample(x_train,y_train_label):
    smo = SMOTE(random_state=42)
    x_smo, y_smo = smo.fit_resample(x_train, y_train_label)
    param = {'C':np.logspace(-3,3,7)}
    svm=LinearSVC(penalty='l1', dual=False,random_state=42)
    svm_param=GridSearchCV(svm,param,cv=10)
    svm_param.fit(x_smo, y_smo)
    c=svm_param.best_params_['C']
    score=svm_param.best_score_
    return c, score


In [6]:
y_train_family, y_train_genus, y_train_species=y_train["Family"], y_train["Genus"], y_train["Species"]
y_test_family, y_test_genus, y_test_species=y_test["Family"], y_test["Genus"], y_test["Species"]

c_family,score_family=Svm_Resample(x_train,y_train_family)
print("----------paramater of Family----------")
print("C_family=",c_family,"score_family=",score_family)
#print("scorescore_family)
c_genus,score_genus=Svm_Resample(x_train,y_train_genus)
print("----------paramater of Genus----------")
print("C_genus=",c_genus,"score_genus=", score_genus)
c_species,score_species=Svm_Resample(x_train,y_train_species)
print("----------paramater of Species----------")
print("C_species=",c_species,"score_species=", score_species)



----------paramater of Family----------
C_family= 10.0 score_family= 0.9503747511030847
----------paramater of Genus----------
C_genus= 100.0 score_genus= 0.9571408789460379
----------paramater of Species----------
C_species= 100.0 score_species= 0.961749080506743


In [16]:
from sklearn.metrics import hamming_loss,accuracy_score

smo_f = SMOTE(random_state=42)
x_smo_f, y_smo_f = smo_f.fit_resample(x_train, y_train_family)
svm_family=LinearSVC(penalty='l1', dual=False,C=c_family).fit(x_smo_f,y_smo_f)
y_predict_family=pd.Series(svm_family.predict(x_test))
hamming_loss_family =hamming_loss(y_predict_family, y_test_family)
exact_match_family= accuracy_score(y_test_family, y_predict_family)
print("----------Family---------")
print("hamming loss of Family:",hamming_loss_family)
print("exact match of Family:",exact_match_family)

smo_g = SMOTE(random_state=42)
x_smo_g, y_smo_g = smo_g.fit_resample(x_train, y_train_genus)
svm_genus=LinearSVC(penalty='l1', dual=False,C=c_genus).fit(x_smo_g,y_smo_g)
y_predict_genus=pd.Series(svm_genus.predict(x_test))
hamming_loss_genus =hamming_loss(y_predict_genus, y_test_genus)
exact_match_genus= accuracy_score(y_test_genus, y_predict_genus)
print("----------Genus---------")
print("hamming loss of genus:",hamming_loss_genus)
print("exact match of Genus:",exact_match_genus)

smo_s = SMOTE(random_state=42)
x_smo_s, y_smo_s = smo_s.fit_resample(x_train, y_train_species)
svm_species=LinearSVC(penalty='l1', dual=False,C=c_species).fit(x_smo_s,y_smo_s)
y_predict_species=pd.Series(svm_species.predict(x_test))
hamming_loss_species =hamming_loss(y_predict_species, y_test_species)
exact_match_species= accuracy_score(y_test_species, y_predict_species)
print("----------Species---------")
print("hamming loss of species:",hamming_loss_species)
print("exact match of species:",exact_match_species)



----------Family---------
hamming loss of Family: 0.09078276980083372
exact match of Family: 0.9092172301991662
----------Genus---------
hamming loss of genus: 0.09402501157943492
exact match of Genus: 0.9059749884205651
----------Species---------
hamming loss of species: 0.03890690134321445
exact match of species: 0.9610930986567855


Gaussian SVM:  
Hamming loss of Family: 0.006021306160259379;exact match of Family: 0.9939786938397406  
Hamming loss of genus: 0.009263547938860583;exact match of Genus: 0.9907364520611394  
Hamming loss of species: 0.009726725335803613;exact match of species: 0.9902732746641963  
  
L1-penalized SVMs  
amming loss of Family: 0.07364520611394164;exact match of Family: 0.9263547938860583  
hamming loss of genus: 0.05974988420565076;exact match of Genus: 0.9402501157943493  
hamming loss of species: 0.03844372394627142;exact match of species: 0.9615562760537286  
  
Using SMOTE  
hamming loss of Family: 0.09078276980083372;exact match of Family: 0.9092172301991662  
hamming loss of genus: 0.09402501157943492;exact match of Genus: 0.9059749884205651  
hamming loss of species: 0.03890690134321445;exact match of species: 0.9610930986567855  
  
From above, the Gaussian SVM performs the best!  